In [10]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
# etc
import os
import pandas as pd
from tqdm import tqdm
import unicodedata

pypdf 이건 x

표가 많기 때문에

PyMuPDFLoader 나 PDFPlumberLoader를 쓰고

표를 이미지로 불러오기도 하는데 이런경우 추가적인 전처리가 필요함

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

def get_embedding():
    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-small',
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': True})
    return embeddings

def normalize_string(s):
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for i, doc in enumerate(docs):
        #context += f"Document {i+1}\n"
        context += doc.page_content
        context += '\n\n'
    return context

In [12]:

def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    bm = BM25Retriever.from_texts(to_vectorize)
    knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss, bm, #knn
    
def make_db(df):
    # Create a new FAISS database
    # pdf reader
    documents = []
    pdf_files = df['Source_path'].unique()
    for pdf_file in pdf_files:
        pdf_loader = PDFPlumberLoader(pdf_file)
        pdf_documents = pdf_loader.load()
        for pdf_document in pdf_documents:
            pdf_document.page_content = pdf_document.page_content.replace("\x07","")
        documents.extend(pdf_documents)
    # 정규화
    for doc in documents:
        doc.page_content = normalize_string(doc.page_content)
    chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=60)
    chunks = chunk_splitter.split_documents(documents)
    print(f"Total number of chunks: {len(chunks)}")
    # FAISS DB 만들기
    faiss = FAISS.from_documents(chunks, embedding=get_embedding())
    bm =  BM25Retriever.from_documents(chunks)
    #knn = KNNRetriever.from_documents(chunks, embeddings=get_embedding())
    return faiss, bm, # knn


In [13]:
# train_faiss_db, train_bm_retrievier, knn_retriever = make_db(train_df) 
# test_faiss_db, test_bm_retrievier, test_knn_retriever = make_db(test_df)
# fewshot_faiss_db, fewshot_bm_retrievier, fewshot_knn_retriever = fewshot_db(train_df)

In [14]:
# train_k = 1
# train_bm_retrievier.k = train_k
# knn_retriever.k = train_k
# faiss_retriever = train_faiss_db.as_retriever(search_kwargs={'k':train_k} )
# train_ensemble_retriever = EnsembleRetriever(
#     retrievers=[train_bm_retrievier,knn_retriever, faiss_retriever], weights=[0.3,0.3, 0.4]
# )

# test_k = 3
# test_bm_retrievier.k = test_k
# test_knn_retriever.k = test_k
# test_faiss_retriever = test_faiss_db.as_retriever(search_kwargs={'k':test_k} )
# test_ensemble_retriever = EnsembleRetriever(
#     retrievers=[test_bm_retrievier,test_knn_retriever, test_faiss_retriever], weights=[0.3,0.3, 0.4]
# )

# fewshot_k = 2
# fewshot_bm_retrievier.k = fewshot_k
# fewshot_knn_retriever.k = fewshot_k
# fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_kwargs={'k':fewshot_k} )
# fewshot_ensemble_retriever = EnsembleRetriever(
#     retrievers=[fewshot_bm_retrievier,fewshot_knn_retriever, fewshot_faiss_retriever], weights=[0.3,0.3, 0.4]
# )


In [15]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.4,
        do_sample=True,
        top_p = 0.6,
        repetition_penalty=1.1,
        return_full_text=False,
        max_new_tokens=512,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.95s/it]


In [16]:
def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def fewshot_ex(fewshot_retriever, train_retriever, q, verbose=False):
    query = normalize_string(q)
    fewshot_results = fewshot_retriever.invoke(normalize_string(query)) #Document(page_content='중소벤처기업부_창업사업화지원\n\n창업사업화지원의 사업목적은 무엇인가?\n\n창업사업화지원의 사업목적은 창업기업의 성장단계별, 초격차 분야별, 글로벌화 지원체계를 구축‧운영하여 혁신 기술창업을 활성화하고 창업기업 성장 및 생존율 제고하는 것이다.')
    fewshot_str = ""
    for result in fewshot_results:
        result = result.page_content.split('\n\n')
        buff_str = "\n###\n"
        buff_str += "<|start_header_id|>user<|end_header_id|>\n"
        question = result[1]
        buff_str += f"Question\n{question}\n\n"
        if train_retriever is not None:
            buff_str += f"Context\n"
            docs = train_retriever.invoke(normalize_string(question))
            if verbose:
                print("Fewshot Q |",len(docs),"|",question)
            buff_str += format_docs(docs)
            buff_str += "<eot_id>"
        buff_str += f"<|start_header_id|>assistant<|end_header_id>\n{result[2]}<|eot_id|>"
        fewshot_str += buff_str    
        
    return fewshot_str

def run (train,test,fewshot,dataset,llm,verbose=False):
    results = []
    for i, row in (dataset.iterrows()):

        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
###
You are the financial expert who helps me with my financial information Q&As.
You earn 10 points when you answer me and follow the rules and lose 7 points when you don't.
Here are some rules you should follow.
- Please use contexts to answer the question.
- Please your answers should be concise.
- Please answers must be written in Korean.
- Please answer the question in 1-3 sentences.
- You might be asked multiple questions. You must answer them all.
- Step by step, when calculating the answer.

- Use the three examples below to learn how to follow the rules and reference information in context.<|eot_id|>
"""
        question = row['Question']        
        if verbose:
            print(f"====={i}/{len(dataset)}{'='*255}")
            print("Question: ", question)
        
        fewshot_str = fewshot_ex(fewshot, train, question,verbose)
        full_template += fewshot_str
        full_template += "\n###\n"

        contexts = test.invoke(normalize_string(question))
        if verbose:
            print("Context Number|",len(contexts),"|")
        contexts = format_docs(contexts)
        full_template += """<|start_header_id|>user<|end_header_id|>\nQuestion\n{input}\n\n"""
        full_template += f"""Context\n{contexts}<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>\n"""
        
        prompt = PromptTemplate.from_template(full_template)
        qa_chain = (
        {
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke(input=question)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print("=====Answer=====\n", results[-1]['Answer'])
            if row['Answer'] is not None:
                print("=====REAL Answer=====\n",row['Answer'])
    return results

In [17]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())
    
    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):
    
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    
    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    
    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)
    
    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

In [18]:
from sklearn.model_selection import KFold
import copy

weight = [0.5,0.5]
train_faiss_db, train_bm_retrievier = make_db(train_df) 

train_k = 1
train_bm_retrievier.k = train_k
#knn_retriever.k = train_k
faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, faiss_retriever], weights=weight
)

test_bm_retrievier = copy.deepcopy(train_bm_retrievier)
test_k = 3
test_bm_retrievier.k = test_k
#test_knn_retriever.k = test_k
test_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':test_k} )
test_ensemble_retriever = EnsembleRetriever(
    retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
)

fewshot_k = 2

k_folds = 4
fold_results = []
kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
    fold_result = []
    train_set = train_df.iloc[train_index]
    val_set = train_df.iloc[val_index]
    
    
    fewshot_faiss_db, fewshot_bm_retrievier = fewshot_db(train_set)

    
    fewshot_bm_retrievier.k = fewshot_k
    fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_kwargs={'k':fewshot_k} )
    fewshot_ensemble_retriever = EnsembleRetriever(
        retrievers=[fewshot_bm_retrievier, fewshot_faiss_retriever], weights=weight
    )
    pred = run(train_ensemble_retriever, test_ensemble_retriever, fewshot_ensemble_retriever, val_set, llm, verbose=True)
    result = pd.DataFrame()
    result['pred'] = [result['Answer'] for result in pred]
    val_set.index = range(len(val_set))
    result['gt'] = val_set['Answer']
        
    result = calculate_average_f1_score(result['gt'], result['pred'])
    print(result)
    fold_results.append(result)
    break

Total number of chunks: 4072


c:\ProgramData\anaconda3\envs\llm_project\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Loaded Fewshot Set: 372
=====1/124===============================================================================================================================================================================================================================================================
Question:  2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?
Fewshot Q | 2024년 중앙정부 재정체계는 어떻게 구성되어 있나요? |Fewshot Context Number| 2 |
Fewshot Q | 2024년 성과지표의 측정여부에 따라 정량지표는 몇 개로 구성되어 있나요? |Fewshot Context Number| 1 |
Fewshot Q | 2024년 예산 기준 특별회계는 얼마이며, 중앙정부 총지출 대비 어느 정도의 비율을 차지하는가? |Fewshot Context Number| 1 |
Context Number| 5 |
=====Answer=====
 2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 특별회계 81.7조원으로 총 438.3조원으로 구성되어 있습니다.
=====REAL Answer=====
 2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성되어 있습니다.
=====4/124=====================================================================================================================================================================================================================

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


=====Answer=====
 2007년 시행된 국가재정법에서 추경 편성 사유로 명시된 사항은 다음과 같습니다.

1. 전쟁이나 대규모 재해가 발생한 경우
2. 경기 침체, 대량 실업, 남북 관계의 변화, 경제 협력과 무역의 변화로 발생하는 국가 세입의 감소
=====REAL Answer=====
 국가재정법에서 추경 편성 사유로 명시된 사항은 전쟁이나 대규모 재해가 발생한 경우, 경기침체, 대량 실업, 남북 관계의 변화, 경제협력과 같은 대내외 여건에 중대한 변화가 발생하였거나 발생할 우려가 있는 경우, 법령에 따라 국가가 지급하여야 하는 지출이 발생하거나 증가하는 경우에 한정되어 있습니다.
=====41/124===============================================================================================================================================================================================================================================================
Question:  2024년 교육재정 교부금의 규모는 얼마이고, 전년 대비 추이는 어떠한가?
Fewshot Q | 국고보조사업 규모는 2024년에 얼마이며 전년 대비 어떤 증가율을 보이는가? |Fewshot Context Number| 2 |
Fewshot Q | 2024년의 교육재정교부금에서 고교 무상교육에 사용한 규모는? |Fewshot Context Number| 2 |
Fewshot Q | 총사업비 관리대상사업의 규모는 2023년 몇 조원이며, 전년 대비 얼마나 증가했나요? |Fewshot Context Number| 2 |
Fewshot Q | 2024년 교육분야 재정투자 규모는 전년 대비 어떻게 변화했는가? |Fewshot Context Number| 1 |
Context Number| 

py pdf 0.663
ptmu 0.6740735793987513

In [ ]:
from save_module import save


# fewshot_faiss_db, fewshot_bm_retrievier, fewshot_knn_retriever = fewshot_db(train_df)
# fewshot_k = 2
# fewshot_bm_retrievier.k = fewshot_k
# fewshot_knn_retriever.k = fewshot_k
# fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_kwargs={'k':fewshot_k} )
# fewshot_ensemble_retriever = EnsembleRetriever(
#         retrievers=[fewshot_bm_retrievier,fewshot_knn_retriever, fewshot_faiss_retriever], weights=[0.3,0.3, 0.4]
#     )
weight = [0.5,0.5]
train_faiss_db, train_bm_retrievier, knn_retriever = make_db(train_df) 
test_faiss_db, test_bm_retrievier, test_knn_retriever = make_db(test_df)
fewshot_faiss_db, fewshot_bm_retrievier, fewshot_knn_retriever = fewshot_db(train_df)

train_k = 1
train_bm_retrievier.k = train_k
#knn_retriever.k = train_k
faiss_retriever = train_faiss_db.as_retriever(search_kwargs={'k':train_k} )
train_ensemble_retriever = EnsembleRetriever(
    retrievers=[train_bm_retrievier, faiss_retriever], weights=weight
)

test_k = 3
test_bm_retrievier.k = test_k
#test_knn_retriever.k = test_k
test_faiss_retriever = test_faiss_db.as_retriever(search_kwargs={'k':test_k} )
test_ensemble_retriever = EnsembleRetriever(
    retrievers=[test_bm_retrievier, test_faiss_retriever], weights=weight
)

fewshot_k = 2
fewshot_bm_retrievier.k = fewshot_k
#fewshot_knn_retriever.k = fewshot_k
fewshot_faiss_retriever = fewshot_faiss_db.as_retriever(search_kwargs={'k':fewshot_k} )
fewshot_ensemble_retriever = EnsembleRetriever(
    retrievers=[fewshot_bm_retrievier, fewshot_faiss_retriever], weights=weight
)

results = run(train_ensemble_retriever, test_ensemble_retriever, fewshot_ensemble_retriever, test_df, llm, verbose=True)
save(results)

Total number of chunks: 4209
